# Dataset

이번 과제의 주제는 "주어진 인물 사진을 요구사항에 맞게 분류하는 것" 입니다.  
따라서 가장 먼저 각 클래스별로 분포를 파악하는 것이 중요하다고 생각했습니다.  
각 클래스는 아래와 같습니다.

분류1: 안경쓰지 않은 남자 성인  
분류2: 안경쓴 남자 성인  
분류3: 남자 아동  
분류4: 안경쓰지 않은 여자 성인  
분류5: 안경쓴 여자 성인  
분류6: 여자 아동  

train set의 각 클래스별 개수는 1723, 601, 230, 2568, 374, 354개였고, 각 클래스의 데이터 불균형이 심했습니다.

In [ ]:
import os
import shutil
import glob
import pandas as pd

DATA_PATH = './label'
IMG_PATH = './faces_images'
TRAIN_PATH = './data/train'
TEST_PATH = './data/test'

for i in range(1,7):
    os.mkdir(TRAIN_PATH + '/' + str(i) + '/')
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train_vision.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test_vision.csv'))

# data augmentation을 염두에 두고 클래스별로 directory를 생성하여 나눴습니다.
# train data move
for i in range(len(df_train)):
    origin_path = os.path.join(IMG_PATH, df_train['filename'][i])
    new_path = TRAIN_PATH + '/' + str(df_train['label'][i]) + '/' + df_train['filename'][i]
    shutil.move(origin_path, new_path)
    
# test data move
for i in range(len(df_test)):
    origin_path = os.path.join(IMG_PATH, df_test['filename'][i])
    new_path = os.path.join(TEST_PATH, df_test['filename'][i])
    shutil.move(origin_path, new_path)

# 각 폴더 개수세기
for i in range(1,7):
    img_cnt = len(glob.glob1(os.path.join(TRAIN_PATH, str(i)), '*.png'))
    print(imag_cnt)

# train, validation split
data_dir = './data/train/'
val_dir = './data/val/'
ref = 1
dir_len = len(os.listdir(val_dir))

if dir_len != 6:
    for i in range(1,7):
        os.mkdir(val_dir + str(i))

for k in range(1,7):
    root_dir = os.path.join(data_dir, str(k))
    output_dir = os.path.join(val_dir, str(k))
    print(len(os.listdir(root_dir)))
    print(len(os.listdir(output_dir)))
    for root, dirs, files in os.walk(root_dir):
        number_of_files = len(os.listdir(root)) 
        if number_of_files > ref:
            ref_copy = int(round(0.2 * number_of_files))
            for i in range(ref_copy):
                chosen_one = random.choice(os.listdir(root))
                file_in_track = root
                file_to_copy = file_in_track + '/' + chosen_one
                if os.path.isfile(file_to_copy) == True:
                    shutil.move(file_to_copy,output_dir)
        else:
            for i in range(len(files)):
                track_list = root
                file_in_track = files[i]
                file_to_copy = track_list + '/' + file_in_track
                if os.path.isfile(file_to_copy) == True:
                    shutil.move(file_to_copy,output_dir)
    print(len(os.listdir(root_dir)))
    print(len(os.listdir(output_dir)))

## Load library

In [1]:
import os
import warnings
import numpy as np 
import pandas as pd
import multiprocessing
from pathlib import Path

from keras import backend as K
warnings.filterwarnings(action='ignore')
K.image_data_format()
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2
from keras.applications.xception import Xception, preprocess_input
from keras.applications.vgg16 import VGG16

# functional
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.merge import concatenate, add

Using TensorFlow backend.


## Path

In [2]:
# 이미지 파일 경로
data_dir = Path('./data/aug')
train_dir = data_dir / 'train'
val_dir = data_dir / 'val'
test_dir = Path('./data')

# prediction 파일 경로
submit_dir = Path('./submit')

# save model 경로
save_dir = Path('./savemodel')

# csv 파일 경로
label_dir = Path('./label')
train_path = label_dir / 'train_vision.csv'
test_path = label_dir / 'test_vision.csv'

In [3]:
params = {
    # Generator Parameter
    'random_state': 42,
    # 좌우 반전
    'horizontal_flip': True,
    # 회전하는 최대 각도
    'rotation_range': 20,
    # 좌우로 이동할 최대 비율
    'width_shift_range': 0.20,
    # 상하로 이동할 최대 비율
    'height_shift_range': 0.20,
    # 확대 및 축소 최대 비율
    'zoom_range': 0.20,
    # 밝기 조절 최소, 최대 비율의 범위
    'brightness_range': (0.7, 1.3),

    # Model Parameter
    'img_size': (128, 128),
    'input_shape': (128, 128, 3),
    'batch_size': 48,
    'epochs': 100,
}

### Keras는 ImageDataGenerator를 사용하여 간편하게 data augmentation 효과를 낼 수 있습니다.

baseline을 구축할 때는 사용했지만, data augmentation을 진행한 후로 ImageDataGenerator는 데이터를 불러오는 용도로만 사용했습니다.

In [4]:
def make_generator(params, train_path, val_path):
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input)
#         rotation_range=params['rotation_range'],
#         width_shift_range=params['width_shift_range'],
#         height_shift_range=params['height_shift_range'],
#         zoom_range=params['zoom_range'],
#         horizontal_flip=params['horizontal_flip'],
#         brightness_range=params['brightness_range'])

    validation_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input)

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=params['img_size'],
        color_mode='rgb',
        class_mode='categorical',
        batch_size=params['batch_size'],
        seed=params['random_state'])

    validation_generator = validation_datagen.flow_from_directory(
        val_path,
        target_size=params['img_size'],
        color_mode='rgb',
        class_mode='categorical',
        batch_size=params['batch_size'],
        shuffle=False)
    
    return train_generator, validation_generator

train_generator, validation_generator = make_generator(params, str(train_dir), str(val_dir))

Found 29946 images belonging to 6 classes.
Found 1171 images belonging to 6 classes.


In [5]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0 :
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

# Model
제가 실험한 방식은 다음과 같습니다.
1. 가장 간단한 모델로 baseline을 구축한다.
    - 아주 간단함에도 불구하고 public score 기준 81.5점이 나왔습니다.  


2. Imagenet 대회에서 좋은 성능을 보인 모델을 사용해본다.
    - Xception, ResNet, VGGNet, EfficientNet 등을 사용했습니다.  
    - 각각의 모델에 대해서 fine tuning을 진행한 결과 Xception 모델이 가장 높은 점수를 얻을 수 있었습니다.  


3. 각 모델의 특징을 파악하고 functional model을 구축한다.
    - VGGNet이 feature를 잘 추출하고, ResNet이 residual block 덕분에 분류를 잘한고 생각을 해서  
    VGGNet의 Fully Connected Layer 부분에 residual block을 추가하는 functional 모델을 구축했습니다.  
    - 결과적으로는 학습이 잘 안돼서 Xception모델과 baseline모델을 사용했습니다.

In [21]:
#### baseline model #####

model = None
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=params['input_shape']))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(2048, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax', kernel_initializer='he_normal'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 58, 58, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 27, 27, 32)       

In [12]:
##### main model #####

cnn_model = Xception(include_top=False, weights=None, input_shape=params['input_shape'])
model = Sequential()
# imagenet 대회를 위한 모델들은 보통 (220,220) ~ (300,300) 사이의 input size에 최적화 되어있어서
# interpolation하여 input size를 (256,256)가 되도록 진행했습니다.
# 결과적으로는 interploation을 적용해도 점수 향상에는 도움이 되지 않고, 연산량만 증가하여 이후에는 사용하지 않았습니다.
# model.add(UpSampling2D(size=(2,2), input_shape=params['input_shape'], interpolation='bilinear'))
model.add(cnn_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation='softmax', kernel_initializer='he_normal'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 4, 4, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 12294     
Total params: 20,873,774
Trainable params: 20,819,246
Non-trainable params: 54,528
_________________________________________________________________


In [22]:
###### functional model ######

inputs = Input(shape=(128,128,3))
upsam = UpSampling2D(size=(2,2), input_shape=(128,128,3), interpolation='bilinear')(inputs)
x1 = Conv2D(64, kernel_size=3, activation='relu', input_shape=(256,256,3), padding='same')(upsam)
x1 = Conv2D(64, kernel_size=3, activation='relu', padding='same')(x1)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)

x2 = Conv2D(128, kernel_size=3, activation='relu', padding='same')(x1)
x2 = Conv2D(128, kernel_size=3, activation='relu', padding='same')(x2)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)

x3 = Conv2D(256, kernel_size=3, activation='relu', padding='same')(x2)
x3 = Conv2D(256, kernel_size=3, activation='relu', padding='same')(x3)
x3 = Conv2D(256, kernel_size=3, activation='relu', padding='same')(x3)
x3 = MaxPooling2D(pool_size=(2, 2))(x3)

x4 = Conv2D(512, kernel_size=3, activation='relu', padding='same')(x3)
x4 = Conv2D(512, kernel_size=3, activation='relu', padding='same')(x4)
x4 = Conv2D(512, kernel_size=3, activation='relu', padding='same')(x4)
x4 = MaxPooling2D(pool_size=(2, 2))(x4)

x5 = Conv2D(512, kernel_size=3, activation='relu', padding='same')(x4)
x5 = Conv2D(512, kernel_size=3, activation='relu', padding='same')(x5)
x5 = Conv2D(512, kernel_size=3, activation='relu', padding='same')(x5)
x6 = MaxPooling2D(pool_size=(2, 2))(x5)

x7 = GlobalAveragePooling2D()(x6)

fc1 = Dense(2048, activation='relu')(x7)
dr = Dropout(0.25)(fc1)
fc2 = Dense(512, activation='relu')(dr)
dr = Dropout(0.25)(fc2)

fc3 = Dense(2048, activation='relu')(fc2)
fc4 = Dense(2048, activation='relu')(fc3)
add_layer = add([fc1, fc4])

fc5 = Dense(2048, activation='relu')(add_layer)
fc6 = Dense(2048, activation='relu')(fc5)
add_layer2 = add([add_layer, fc6])

fc7 = Dense(2048, activation='relu')(add_layer2)
fc8 = Dense(2048, activation='relu')(fc7)
add_layer3 = add([add_layer2, fc8])
outputs = Dense(6, activation='softmax', kernel_initializer='he_normal')(add_layer3)

model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])
model.summary()
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
up_sampling2d_1 (UpSampling2D)  (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 256, 256, 64) 1792        up_sampling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_8[0][0]                   
____________________________________________________________________________________________

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
#               optimizer=optimizers.Adadelta(),
              metrics=['acc'])

In [17]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# model을 저장할 때 acc, val_loss, val_acc를 함께 저장할 수 있도록 했습니다.
filepath = str(save_dir / 'Xception_adam_ep{epoch:03d}_acc-{acc:.4f}_vloss-{val_loss:.4f}_vacc-{val_acc:.4f}.h5')
# 학습을 이어서 진행할 경우 모델을 불러올 수 있습니다.
# model = load_model(str(save_dir / 'res152_aug8+real8_val2_ep040_acc-0.9854_vloss-1.7443_vacc-0.9078.h5'))

# val_acc를 관찰함으로써 성능이 향상되었을 때만 모델을 저장하도록 할 수 있습니다.
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True)
# 일정 구간동안 성능이 올라가지 않으면 학습을 조기 종료 시킬 수 있습니다.
# earlystop = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

# callbacks = [checkpoint, earlystop]
callbacks = [checkpoint]
model.save(filepath)

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = get_steps(train_generator.samples, params['batch_size']),
    validation_data = validation_generator, 
    validation_steps = get_steps(validation_generator.samples, params['batch_size']),
    callbacks = callbacks,
    # 학습을 이어서 진행할 경우 초기 epoch을 지정할 수 있습니다.
    # initial_epoch = 22,
    epochs = params['epochs'])

# Predict and Make submission  
### Model predict

In [8]:
from keras.models import load_model

model_filename = 'fix_aug8+real8_val2_ep045_acc-0.9954_vloss-3.3429_vacc-0.9180.h5'
model_filepath = str(save_dir / model_filename)
csv_filename = model_filename + '.csv'

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    str(test_dir),
    classes=['test'],
    target_size=params['img_size'],
    color_mode='rgb',
    class_mode='categorical',
    batch_size=params['batch_size'],
    shuffle=False)

model = load_model(model_filepath)

prediction = model.predict_generator(
    generator=test_generator,
    steps = get_steps(test_generator.samples, params['batch_size']),
    verbose=1)

file_name = test_generator.filenames
for i in range(len(file_name)):
    file_name[i] = file_name[i].split('/')[-1]

Found 2000 images belonging to 1 classes.
42/42 [==============================] - 12s 277ms/step


### Make submission

In [9]:
predicted_class_indices = np.argmax(prediction, axis=1)

# Generator class dictionary mapping
train_generator, _ = make_generator(params,str(train_dir),str(val_dir))
labels = (train_generator.class_indices)
labels = dict((v, k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
submission = pd.read_csv(str(test_path))
submission = pd.DataFrame(submission)

df = pd.DataFrame(file_name, columns=['filename']) 
df2 = pd.DataFrame(predictions, columns=['prediction'])
df['prediction'] = df2

submission = pd.merge(submission, df, on="filename")

submission = submission[['prediction']]
submission.to_csv(str(submit_dir / csv_filename), index=False)
submission.tail()

Found 29946 images belonging to 6 classes.
Found 1171 images belonging to 6 classes.


,prediction
1995,1
1996,1
1997,5
1998,4
1999,4


## Confusion matrix
각각의 모델이 실제 클래스를 어떤 클래스로 분류하는지 비교하기 위해서 confusion matrix를 관찰했습니다.  
test set에는 label이 없기 때문에 validation set을 통해 비교하였습니다.  
결과적으로 남성과 여성을 구별하는 것은 잘 하지만, 남자 아동과 여자 아동을 분류하는 것을 잘 못하고 있는 것을 알게 되었습니다.

In [11]:
model_filename = 'fix_aug8+real8_val2_ep008_acc-0.9622_vloss-1.1711_vacc-0.9026.h5'
model_filepath = str(save_dir / model_filename)
csv_filename = model_filename + '.csv'

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    str(val_dir),
#     classes=['test'],
    target_size=params['img_size'],
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False)

model = load_model(model_filepath)

train_generator, _ = make_generator(params,str(train_dir),str(val_dir))
prediction = model.predict_generator(
    generator=test_generator,
    steps = get_steps(test_generator.samples, 32),
    verbose=1)

file_name = test_generator.filenames
for i in range(len(file_name)):
    file_name[i] = file_name[i][2:]

predicted_class_indices = np.argmax(prediction, axis=1)

# Generator class dictionary mapping
labels = (train_generator.class_indices)
labels = dict((v, k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
submission = pd.read_csv(str(train_path))
submission = pd.DataFrame(submission)
df = pd.DataFrame(file_name, columns=['filename']) 
df2 = pd.DataFrame(predictions, columns=['prediction'])
df['prediction'] = df2
submission = pd.merge(submission, df, on="filename")
submission = submission.dropna(axis=0)
csv_filename = 'prediction' + model_filename + '.csv'
submission.to_csv(str(submit_dir / csv_filename), index=False)
submission.tail()

Found 1171 images belonging to 6 classes.
37/37 [==============================] - 7s 183ms/step


,filename,label,prediction
1166,face_6455.png,1,1
1167,face_6245.png,4,4
1168,face_452.png,1,1
1169,face_1334.png,3,3
1170,face_7020.png,4,4


In [12]:
submission = submission[['label','prediction']]
label = submission['label'].values.tolist()
prediction = submission['prediction'].values.tolist()

for i in range(len(label)):
    label[i] = str(label[i])
conf_matrix = [[0] * 6 for _ in range(6)]
for i in range(len(label)):
    conf_matrix[int(label[i])-1][int(prediction[i])-1] += 1

In [13]:
conf_matrix

[[327, 1, 2, 15, 0, 0],
 [0, 119, 0, 0, 1, 0],
 [1, 0, 42, 0, 0, 3],
 [2, 0, 2, 502, 1, 7],
 [0, 0, 0, 0, 75, 0],
 [0, 0, 3, 2, 0, 66]]

# Data Augmentation  
Data augmentation에 사용한 기법은 아래와 같습니다.
1. brightness
2. sharpening
3. gausian blur
4. saturation
5. salt
6. rotation
7. flip

In [ ]:
from pathlib import Path
import os
import cv2
import numpy as np

Extension=".png"
data_dir = Path('./data/aug')
raw_dir = data_dir / 'train'
aug_dir = data_dir / 'train'
dir_list = os.listdir(str(raw_dir))

def add_light(image_file):
    image=cv2.imread(image_file)
    gamma = np.random.randint(7,20) / 10 #(0.7, 2.0)
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    image=cv2.LUT(image, table)
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)

def sharpen_image(image_file):
    image=cv2.imread(image_file)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    image = cv2.filter2D(image, -1, kernel)
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)

def gausian_blur(image_file, blur):
    image=cv2.imread(image_file)
    image = cv2.GaussianBlur(image,(5,5),blur)
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)

def saturation_image(image_file):
    image=cv2.imread(image_file)
    saturation = np.random.randint(0, 50)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    v = image[:, :, 2]
    v = np.where(v <= 255 - saturation, v + saturation, 255)
    image[:, :, 2] = v
    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)
    
def salt_image(image_file):
    image=cv2.imread(image_file)
    p = 0.5
    a = np.random.rand() / 50
    noisy = image
    num_salt = np.ceil(a * image.size * p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    noisy[coords] = 1
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)

def rotate_image(image_file):
    image=cv2.imread(image_file)
    deg = np.random.uniform(-30,30)
    rows, cols,c = image.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), deg, 1)
    image = cv2.warpAffine(image, M, (cols, rows))
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)

def flip_image(image_file):
    image=cv2.imread(image_file)
    dir = 1
    image = cv2.flip(image, dir)
    cv2.imwrite(str(Folder_name) + '/' + file_no + str(i) + Extension, image)

for dirname in dir_list:
    LOAD_PATH = raw_dir / dirname
    Folder_name = aug_dir / dirname
    file_list = os.listdir(str(LOAD_PATH))
    # 각 클래스 별로 4400장씩 생성했고, 이를 통해 데이터의 균형을 맞출 수 있었습니다.
    cnt = 4400 // len(file_list)
    print('dir:', dirname)
    for i in range(0, cnt):
        print(i)
        for filename in file_list:
            image_file = str(Folder_name) + '/' + filename
            file_no = filename[5:-4] + '_'    
            
            add_light(image_file)
            image_file = str(Folder_name) + '/' + file_no + str(i) + Extension
            rotate_image(image_file)
            if np.random.rand() < 0.5:
                flip_image(image_file)
            if np.random.rand() < 0.2:
                salt_image(image_file)
            if np.random.rand() < 0.3:
                blur = np.random.rand()
                gausian_blur(image_file, blur)
            if np.random.rand() < 0.3:
                saturation_image(image_file)
            if np.random.rand() < 0.3:
                sharpen_image(image_file)

## Majority vote
ensemble 기법 중에 결과를 기반으로 한 majority vote 방식을 사용했습니다.  
각 모델의 prediction을 public score를 기준으로 더했으며, 가장 높은 점수를 얻은 클래스를 택하도록 했습니다.

In [16]:
from pathlib import Path
import numpy as np
import pandas as pd
import os

In [17]:
data_dir = './bestmodel'
file_list = os.listdir(data_dir)
file_list

['90.25_fix_aug8+real8_val2_ep021_acc-0.9872_vloss-5.8906_vacc-0.9103.h5.csv',
 '89.5_aug2_ep061_acc-0.9245_vloss-0.0521_vacc-0.9815.csv',
 '89.5_c7p3sh1z1_adadelta_ep86_vloss-0.452_vacc-0.931.csv']

In [18]:
df_list = [pd.read_csv(os.path.join(data_dir, file), engine='python') for file in file_list]
scores = [float(file.split('_')[0]) for file in file_list]
print(scores)

[90.25, 89.5, 89.5]


In [19]:
n_class = len(pd.unique(df_list[0]['prediction']))
n_class
result = []
for i in range(df_list[0].shape[0]):
    classes = [0] * (n_class + 1)
    for j in range(len(df_list)):
        classes[df_list[j]['prediction'][i]] += scores[j]
    result.append(classes.index(max(classes)))

In [20]:
submission = df_list[0]
submission['prediction'] = result
submission.to_csv(os.path.join(data_dir, 'majority_vote.csv'), index=False)
submission.tail()

,prediction
1995,1
1996,1
1997,5
1998,4
1999,4
